In [14]:
Out_path = r"Out"
Excel_path = r"C:\Users\rimit\Amitabha\Excel.xlsx"

In [15]:
pip install xlwings

Note: you may need to restart the kernel to use updated packages.


In [16]:
pip install pywin32

Note: you may need to restart the kernel to use updated packages.


In [17]:
pip install --upgrade pywin32 --user

Note: you may need to restart the kernel to use updated packages.


In [18]:

import re
import pandas as pd
import openpyxl
from openpyxl import load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows
import shutil
from pathlib import Path
import os
import csv
import win32com.client as win32

# Define patterns
A1 = r'The table below is a sample table1\n'
A2 = r'The table below is a sample table2+\n'
A3 = "Table-1"

B1 = r'The table below is a sample table2\n'
B2 = r'The End of report+\n'
B3 = "Table-2"



def my_table(line1, line2, file_path, NAME):
    heading_pattern = re.compile(line1)
    bottom_line_pattern = re.compile(line2)
    with open(file_path, 'r') as file:
        content = file.read()
    matches = re.finditer(heading_pattern, content)
    tables = []
    for match in matches:
        start = match.end()
        start += content[start:].find('\n')
        start += content[start:].find('\n') + 1
        end_match = bottom_line_pattern.search(content, start)
        if end_match:
            end = end_match.start()
            table_content = content[start:end].strip()
            tables.append(table_content)

    for table in tables:
        table_data = [line.split(',') for line in table.split('\n')]
        df4 = pd.DataFrame(table_data[1:], columns=table_data[0])
        
    #import win32com.client as win32
    
    file_path = Excel_path
    excel = win32.gencache.EnsureDispatch('Excel.Application')
    excel.DisplayAlerts = False
    
    wb = excel.Workbooks.Open(file_path)
    new_sheet_name = NAME
    sheet_names = [sheet.Name for sheet in wb.Sheets]
    
    if new_sheet_name in sheet_names:
        sheet = wb.Sheets(new_sheet_name)
    else:
        sheet = wb.Sheets.Add()
        sheet.Name = new_sheet_name
    for col_num, header in enumerate(df4.columns, start=1):
        sheet.Cells(1, col_num).Value = header
    for row_num, row in enumerate(df4.itertuples(index=False), start=2):
        for col_num, value in enumerate(row, start=1):
            sheet.Cells(row_num, col_num).Value = value
    wb.Save()


In [19]:
# Loop through directory files and populate "Control Panel"
files = os.listdir(Out_path)
Runs = []
A = len(files)
while A > 0:
    Runs.append(A)
    A = A - 1
Runs.reverse()
Out_files = []
s = 26
i = 1
for file in files:
    Out_files.append(f"{Out_path}\\{file}")
    excel = win32.gencache.EnsureDispatch('Excel.Application')
    excel.DisplayAlerts = False
    wb = excel.Workbooks.Open(Excel_path)
    sheet = wb.Sheets('Reference')
    sheet.Cells(s, 3).Value = i
    sheet.Cells(s, 4).Value = file
    s += 2
    i += 1
    # wb.Save()
    # wb.Close()

In [20]:
# Call the functions
for i, j in zip(Out_files, Runs):
    my_table(A1, A2, i, f"{A3}_{j}")
    my_table(B1, B2, i, f"{B3}_{j}")